In [ ]:
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import nest
import itertools

# Introduction
In order to understand the inner workings of neuronal circuits, one ansatz is to build and study computer models of such circuits. In those models we try to create carefully designed reduced models of complex structures, taking into account only what we believe is important to address a certain scientific question. If the reduced model is able to reproduce certain properties of the more complex system, one can hope that analyzing the model further might reveal answers to those questions or generate hypothesis which then can be experimentally tested.
In this introductory part of the NEST project we will investigate a basic model of a cortical circuit. But in order to assess whether the model reasonably reproduces dynamical features of cortical circuits, we need to know how the dynamics in those circuits looks like.
</br>
## Raster plot and instantaneous rate
Given spike trains taken from the observed data of an experimental setups, the first thing one often considers are the spike raster plot and the instantaneous firing rate of the populations.
<img src="images/raster.jpeg" width="600" align="center">
<caption>Spike raster plot of neuronal population in visual cortex of awake mice. Lower parts shows temporal histogram of the spontaneous activity. Figure taken from <a href="https://www.jneurosci.org/content/35/23/8813"> Carrillo-Reid et al. 2015, Journal of Neuroscience. </a><caption>

   
Recording multiple trials from a single neuron, one may also consider the spike raster plots over trials.
<img src="images/raster_2.jpeg" width="500" align="center">
<caption>Raster plot and firing rate of MT neuron in multiple trials given different stimulus parameters. Taken from Dayan and Abbot, Theoretical Neuroscience, p. 39<caption>
    
## Insterspike interval distribution
One commonly also studies the interspike interval (ISI) distribution of neural firing.
</br>
<img src="images/isi.jpeg" width="350" align="center">
<caption> Interspike interval distribution from an MT neuron. Taken from Dayan and Abbot, Theoretical Neuroscience, p. 33 <caption>
    
## Correlation histograms
Finally, we consider auto- and cross-correlation histograms. These are studied to study correlations and synchrony in neural systems under a (variable) time delay.
<img src="images/correlations.jpeg" width="400" align="center">
<caption>Auto- and cross-correlation function for neurons in cat V1. Oscillations are induced via a specific stimulus. Taken from Dayan and Abott, Theoretical Neuroscience, p. 29<caption>
 <img src="images/correlations_2.jpeg" width="400" align="center">   
 <caption>Cross-correlations of several pairs of hippocampus CA1 neurons. The arrows indicate putative connections inferred from the correlation function. Figure taken from <a href="https://www.nature.com/articles/s41467-019-12225-2">Kobayashi et al. 2019, Nature Communications </a><caption>

# NEST - Getting started
## The Brunel network

To learn the basic functionalities of NEST and get a feeling which type question we might address with spiking neural network simulations we will implement the [Brunel network](https://web.stanford.edu/group/brainsinsilicon/documents/BrunelSparselyConnectedNets.pdf).  
</br>
It's a sparse random network consisting of $N$ leaky integrate-and-fire model neuron $N_E$ of which are excitatory and $N_I$ inhibitory. Each neuron receives $C$ randomly chosen connections of which $C_E = \epsilon N_E$ are excitatory and $C_I = \epsilon N_I$ are inhibitory. The coupling strength for excitatory connections is $J$, for inhibitory connections it is $-g\cdot J$. Each neuron in the network is provided with an excitatory Poissonian background spike train whose strength is measured in 
\begin{equation}
    \eta = \frac{\eta_{\mathrm{ext}}}{\eta_{\mathrm{thr}}}.
\end{equation}
Here, $\eta_{\mathrm{thr}}$ denotes the rate at which the average membrane potential of a neuron provided with a Poissonian background spike train reaches the threshold.
</br>
Setting $\gamma = 0.25$, $\rho = 0.2$, typical parameter choices are:
</br>
$N = 10000$, $N_I = \rho \cdot N$, $N_E = (1 - \rho) \cdot N$, $\epsilon = 0.1$, $g = 4$, $\eta = 2\frac{\mathrm{spikes}}{\mathrm{s}}$.
</br>
A suitable size of $J$ depends on the parameters of the model neurons, but it is important that $g \geq 4$.
Otherwise we - without going into much detail - expect that the average input of a neuron in the network to be proportional to
\begin{equation}
    C_E\cdot J - C_I \cdot g \cdot J = \epsilon \cdot N \cdot J \cdot \big( (1-\rho) - g \cdot \rho \big)
\end{equation}
If this quantity is positive, the dynamics of the network will result in a positive feedback loop leading to exploding activity in the network.
</br>
Therefore, for the network to be in a low firing-rate regime - the following relation should hold true:
\begin{equation}
    (1-\rho) - g \cdot \rho \leq 0.
\end{equation}
Solving for $g$ with $\rho = 0.2$ yields the aforementioned constraint.

In [ ]:
# Network parameters
neuron_params = {"C_m": 1.0,
                 "tau_m": 20.,
                 "t_ref": 2.0,
                 "E_L": 0.0,
                 "V_reset": 0.0,
                 "V_m": 0.0,
                 "V_th": 20.}

params = {
    'num_neurons': 12500,  # number of neurons in network
    'rho': 0.2, # fraction of inhibitory neurons
    'eps': 0.1, # probability to establish a connections
    'g': 4.5,  # excitation-inhibition balance
    'eta': 0.9,  # relative external rate
    'J': 0.1,  # postsynaptic amplitude in mV
    'neuron_params': neuron_params,  # single neuron parameters
    'n_rec_ex': 200,  # excitatory neurons to be recorded from
    'n_rec_in': 100,  # inhibitory neurons to be recorded from
    'rec_start': 600.,  # start point for recording spike trains
    'rec_stop': 800.  # end points for recording spike trains
}

# Task
We now implement the Brunel network. Instead of doing it straightforwardly, we will do it in a Python class.
</br>
Though it might seem unnecessary at the moment, it will turn out to be convenient later.
</br>
The [nest documentation](https://nest-simulator.readthedocs.io/en/v3.3/) is your friend.
You might want to have a look at the [tutorial](https://nest-simulator.readthedocs.io/en/v3.3/tutorials/index.html) as well as the documentation of the [poisson generator](https://nest-simulator.readthedocs.io/en/v3.3/models/poisson_generator.html?highlight=poisson), the [spike recorder]( https://nest-simulator.readthedocs.io/en/v3.0/models/spike_recorder.html) and the [connection management](https://nest-simulator.readthedocs.io/en/v3.3/guides/connection_management.html)

In [ ]:
class BrunelNetwork:
    def __init__(self, num_neurons, rho, eps, g, eta, J, neuron_params, n_rec_ex, n_rec_in, rec_start, rec_stop):
        self.num_neurons = num_neurons
        self.num_ex = int((1 - rho) * num_neurons)  # number of excitatory neurons
        self.num_in = int(rho * num_neurons)  # number of inhibitory neurons
        self.c_ex = int(eps * self.num_ex)  # number of excitatory connections
        self.c_in = int(eps * self.num_in)  # number of inhibitory connections
        self.J_ex = J  # excitatory weight
        self.J_in = -g*J  # inhibitory weight
        self.n_rec_ex = n_rec_ex # number of recorded excitatory neurons, both excitatory and inhibitory
        self.n_rec_in = n_rec_in # number of recorded excitatory neurons, both excitatory and inhibitory
        self.rec_start = rec_start
        self.rec_stop = rec_stop
        self.neuron_params = neuron_params  # neuron params
        self.ext_rate = (self.neuron_params['V_th']   # the external rate needs to be adapted to provide enough input
                         / (J * self.c_ex * self.neuron_params['tau_m'])
                         * eta * 1000. * self.c_ex)
        
    def create(self):
        # Create the network
        
        # First create the neurons - the exscitatory and inhibitory populations
        
        ## Your code here

        
        # Then create the external poisson spike generator
        
        ## Your code here

        
        # Then create spike detectors
        
        ## Your code here

        
        
        # Next we connect the excitatory and inhibitory neurons to each other, choose a delay of 1.5 ms
        
        ## Your code here

        
        # Then we connect the external drive to the neurons
        
        ## Your code here
        
        # Then we connect the the neurons to the spike detectors
        # Note: You can use slicing for nest node collections as well
        
        ## Your code here
       
    def simulate(self, t_sim):
        # Simulate the network with specified 
        nest.Simulate(t_sim)
        
    def get_data(self):
        '''
        Return spiking data from simulation
        
        Returns
        -------
        
        spikes_ex : list
            list of list of excitatory spike trains of n_rec_ex neurons
        
        spikes_in : list 
            list of list of inhibitory spike trains of n_rec_in neurons
        '''
        # Define lists to store spike trains in
        spikes_ex = []
        spikes_in = []
        
        ## Your code here
        ## You can get the recorded quantities from the spike recorder with nest.GetStatus
        ## You may loop over the entries of the GetStatus return
        ## you might want to sort the spike times, they are not by default

            
        return spikes_ex, spikes_in
        

In [ ]:
# Helper function to plot spiking activity
def plot_raster_rate(spikes_ex, spikes_in, rec_start, rec_stop, figsize=(9, 5)):
    
    spikes_ex_total = list(itertools.chain(*spikes_ex))
    spikes_in_total = list(itertools.chain(*spikes_in))
    spikes_total = spikes_ex_total + spikes_in_total
    
    n_rec_ex = len(spikes_ex)
    n_rec_in = len(spikes_in)
    
    fig = plt.figure(constrained_layout=True, figsize=figsize)
    gs = fig.add_gridspec(5, 1)
    
    ax1 = fig.add_subplot(gs[:4,0])
    ax2 = fig.add_subplot(gs[4,0])
    
    ax1.set_xlim([rec_start, rec_stop])
    ax2.set_xlim([rec_start, rec_stop])
    
    ax1.set_ylabel('Neuron ID')
    
    ax2.set_ylabel('Firing rate')
    ax2.set_xlabel('Time [ms]')
    
    
    for i in range(n_rec_in):
        ax1.plot(spikes_in[i],
                 i*np.ones(len(spikes_in[i])),
                 linestyle='',
                 marker='o',
                 color='r',
                 markersize=2)
    for i in range(n_rec_ex):
        ax1.plot(spikes_ex[i],
                 (i + n_rec_in)*np.ones(len(spikes_ex[i])),
                 linestyle='',
                 marker='o',
                 color='b',
                 markersize=2)

        
    ax2 = ax2.hist(spikes_ex_total,
                   range=(rec_start,rec_stop),
                   bins=int(rec_stop - rec_start))

    plt.savefig('raster.png')

    time_diff = (rec_stop - rec_start)/1000.
    average_firing_rate = (len(spikes_total)
                           / time_diff
                           /(n_rec_ex + n_rec_in))
    print(f'Average firing rate: {average_firing_rate} Bq')
    

After having implemented the Brunel network we will investigate it's dynamical states.
</br>
As it turns out, the network mainly does exhibit four different dynamical states (c.f. figure 8 from paper):
* a fast oscillatory state with high firing rates and synchronous activity (panel A)
* a fast oscillatory state with lower firing rates and asynchronous activity (panel B)
* a asynchronous and irregular state (panel C)
* a slow oscillatory state with irregular activity (panel D)

<img src="images/brunel-phase.png" width="400" align="center"><caption>Phase diagram showing parameter combinations leading to the different dynamical regimes (Brunel 2000, Fig. 7)</caption>

## Task
Reproduce Figure 8 from the paper and explore the different dynamical regimes - don't think to much about the presentation.

<img src="images/brunel-fig8.png" width="600" align="center">

# Fast oscillations, high firing rates and synchronous regular activity

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})  # Adapt if necessary
# Set params['g']
# Set params['eta']
params['J'] = 0.1
params['rec_start'] = 600.
params['rec_stop'] = 800.

# Your code here

# Fast oscillations, lower firing rates and irregular activity

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
# Set params['g']
# Set params['eta']
params['J'] = 0.1
params['rec_start'] = 600.
params['rec_stop'] = 800.

network = BrunelNetwork(**params)

# Your code here

# Asnchronous and irregular state

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
# Set params['g']
# Set params['eta']
params['J'] = 0.1
params['rec_start'] = 600.
params['rec_stop'] = 800.

# Your code here

# Slow oscillations and irregular activity

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
# Set params['g']
# Set params['eta']
params['J'] = 0.1
params['rec_start'] = 600.
params['rec_stop'] = 800.

# Your code here

So far, we only have changed the relative external rate $\eta$ as well as the excitation-inhibition balance $g$.
</br>
In the example below we change the EPSP $J$ to induce a more synchronous state in the network.
</br> 
But there are also other parameters to tweak! Up to now, we assumed a constant delay. However, in biology the delay is not constant but rather mediated. Which effect does a wide delay distribution have on the network dynamics?
To draw the delays from a distribution, you can use the [internal random number generator of NEST](https://nest-simulator.readthedocs.io/en/v3.3/guides/random_numbers.html#random-examples) when setting the delay parameter of the connection: `nest.random.uniform(min=0., max=4.)`. 

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
# Set params['g']
# Set params['eta']
# Set params['J']
params['rec_start'] = 600.
params['rec_stop'] = 800.

# Your code here

# Spike train statistics
After we got a first glance at the produced spiking data, we will continue investigating the obtained acitivty. We next turn to the inter-spike interval (ISI).
</br>
Two quantities studied most frequently in this context are the ISI distribution (i.e. the probability distribution function of the inter-spike interval) and the coefficient of variation (CV).
</br>
The latter one is defined as
\begin{equation}
    \mathrm{CV} = \frac{\sigma}{\mu}
\end{equation}
where $\mu$ denotes mean and $\sigma$ the standard deviation of the spike train. The CV is a measure for the regularity of the spike train:
* a small CV << 1 implies that relative to the mean the standard deviation is small --> regular firing
* a large CV >> 1 implies that relative to the mean the standard deviation is large --> bursty behaviour
* a CV ~ 1 implies irregular firing

As a matter of fact, the CV of an [exponential ISI distribution](https://en.wikipedia.org/wiki/Exponential_distribution) equals $1$. This is often a basic model for neuronal firing.
</br>
And indeed, due to the memorylesness property of an exponential distribution, i.e. for an exponentially distributed $T$ we have
\begin{equation}
    \mathbb{P}[T> s+t | T > t] = \mathbb{P}[T > s],
\end{equation}
the an spike train with exponentially distributed ISIs is "the most random" possible, since no matter how long one already waited for the next spike, the probability of a spike occuring in the next time interval is independent of that waited time.

# Task
Implement a function to plot the ISI of a single neuron, a function to plot the averaged ISI of the spike times of a list of neurons as well as a function to comput the coefficient of variation.
</br>
After you have done that, you can investigate the ISI of the spiking activity of the Brunel network in the different dyanmical states as well as the CV.

In [ ]:
def single_isi(spike_train, min_val, max_val, num_bins):
    '''
    Function to splot the ISI distribution of a sinle spike train
    
    Parameters:
    
    spike_train : np.ndarray
        spike train
    min_val : float
        minimal value of the range to calculate the distribution over
    max_val : float
        minimal value of the range to calculate the distribution over
    num_bins : int
        number of bins for the histogram of the distribution
    '''
    
    ## Your code here
    ## The numpy.histogram function may come in handy here
    ## Read carefully the documentation regarding the auto bins
    
    
def average_isi(spike_trains, min_val, max_val, num_bins):
    '''
    Plot the average ISI distribution of multiple spike trains
    
    Paramters
    ---------
    
    spike_trains : list of np.ndarrays
        list of spike trains
    min_val : float
        minimal value of the range to calculate the distribution over
    max_val : float
        minimal value of the range to calculate the distribution over
    num_bins : int
        number of bins for the histogram of the distribution  
    '''
    
    ## Your code here
    ## If you want to, you can also calculate the CV here and print it direchtly from the function

In [ ]:
# Now lets have a look at the ISI distribution and CV of the network in different states
# Lets start with the fast oscillations, high firing rates and synchronous regular activity state

We see that the mean of the ISI distribution is much larger than its standard deviation hinting that the activity is of the single neurons is very regular.
</br>
Next we turn to a network configuration that shows irregular activity.

In [ ]:
# Lets initiliaze the network in the highly oscillatory, synchronous state
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
params['g'] = 5
params['eta'] = 1.2
params['J'] = 0.1
params['rec_start'] = 100.
params['rec_stop'] = 1000.

## Your code here

The CV now is substantially higher from which we can conclude that the activity is much more irregular.
</br>
Finally we note that global oscillations in the activity do not necessarily translate in regular activity at the single neuron level.

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
params['g'] = 4.5
params['eta'] = 0.9
params['J'] = 0.5
params['rec_start'] = 100.
params['rec_stop'] = 2000.

## Your code here

# Time resolved correlations
After having studied the rater plot, the firing rate as well as properties of the ISI, we will turn to another tool with which we can study network dynamics: The **correlation functions**.
</br>
Given two neurons $i,j$ we consider their spike trains
\begin{equation}
    \xi_i (t) = \sum_{k} \delta(t-t^{f} _ {i,k}), \quad \xi_j (t) = \sum_{k} \delta(t-t^{f} _ {j,k}).
\end{equation}
Here, $\delta$ denotes the Dirac distribution, $t^{f} _ {i,k}$ the $k$-th spike time of the neuron $i$.
</br>
Given the spike trains as well as a bin-width $h$ we can define the spike count of neuron $i$ as 
\begin{equation}
    x_{i} ^ {h}(t) = \int_{t} ^ {t + h}\mathrm{d}s \: \xi_i (s)
\end{equation}
and similarly for neuron $j$.
</br>
Writing $\langle \rangle$ for the expected value, we define the correlation functions as
\begin{equation}
    c_{ij}(\tau) = \langle x_{i} ^ {h}(t) x_{j} ^ {h}(t+\tau) \rangle.
\end{equation}
Luckily, the spike count correlations function can be calculated by a NEST object, see [here](https://nest-simulator.readthedocs.io/en/v3.0/models/correlomatrix_detector.html?highlight=correlomatrix).
</br>
**Note**: In general, the correlation function does not only depend on a time lag but rather on two time variables $t_1, t_2$,
\begin{equation}
    c_{ij}(t_1, t_2) = \langle x_{i} ^ {h}(t_1) x_{j} ^ {h}(t_2) \rangle.
\end{equation}
However, if the time series are stationary, the correlation function does only depend on the difference $\tau = t_2 - t_1$ leading after substitution to the former definition.

**Task**: Add the ```correlomatrix_detector``` to the network and obtain the spike count covariance. You can find the documentation [here](https://nest-simulator.readthedocs.io/en/v3.0/models/correlomatrix_detector.html?highlight=correlomatrix). The documentation of this function is not very good, so don't worry if you cannot figure it out. I will gladly help you.

In [ ]:
class BrunelNetwork(BrunelNetwork):
    
    def correlomatrix(self, n_rec_corr_ex=10,
                      n_rec_corr_in=10,
                      delta_tau_rel=5,
                      tau_max=40.,
                      t_start=300.,
                      t_rec_corr=2300.):
        self.n_rec_corr_ex = n_rec_corr_ex
        self.n_rec_corr_in = n_rec_corr_in
        self.n_rec_corr = self.n_rec_corr_ex + self.n_rec_corr_in
        self.dt = nest.GetKernelStatus()['resolution']
        self.delta_tau_rel = delta_tau_rel
        self.tau_max = tau_max
        self.t_start = t_start
        self.t_rec_corr = t_rec_corr
        
        # Creat the correlomatrix detector
        
        ## Your code here
        
        # Set the status (using nest.SetStatus) of the correlomatrix_detector
        
        ## Your code here
        
        ## Now you need to connect neuron to he correlomatrix_detector
        ## You as syn_spec, you must set a different receptor_type for each neurons. You can use consecutive numbers (0, 1, ..., N) for that.

            
    def get_correlomatrix(self):
        '''
        Function to get and stack the results from the correlomatrix detector
        
        Returns
        -------
        times : np.ndarry
            time points
        cc_2 : np.ndarray
            correlation function dims=(n_rec_corr, n_rec_corr)
        n_events : np.ndarray
            number of spikes by the neurons
        '''
        
        ## Your code here
        ## Note that you need to stack the negative and positive part
        ## The times array you have to generate by hand, numpy.arange might come in handy here

                
        return times, cc_2, n_spikes

In [ ]:
def average_cc(times, cc, n_rec_corr_ex=10, n_rec_corr_in=10, figsize=(15, 15)):
    '''
    Method to average ee, ie, ei, and ii correlations functions and plot them
    
    Parameters
    ----------
    times : np.ndarray
        time array over which cc is computed
    cc : np.ndarray
        count covariance function returned from nest
    '''
    
    ## Your code here
    ## You may use a nested loop over n_rec_corr and n_rec_corr to add the data for ee, ie, ei, and ii correlations


    ## For plotting you may use grid spec as follows
    fig = plt.figure(constrained_layout=True, figsize=figsize)
    gs = fig.add_gridspec(4, 4)
    
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[1, 0])
    ax4 = fig.add_subplot(gs[1, 1])

    
    ax1.plot(times, c_ee, label='C_ee')
    ax1.legend()
    ax2.plot(times, c_ei, label='C_ei')
    ax2.legend()
    ax3.plot(times, c_ie, label='C_ie')
    ax3.legend()
    ax4.plot(times, c_ii, label='C_ii')
    ax4.legend()
 

# Task
Next, we want to study the correlations functions for the Brunel network in different states.
</br>
Interesting to look at could be
* the asynchronous irregular state  
* the slow oscillatory state  
* the slow oscillatory state with increased weight

How do the correlation functions differ? How does the correlation functions look for very short times? What does this imply?

# Extensions of the Brunel network
So far, we mostly considered a parameter regime which correspondes to the asynchronous irregular or slow oscillatory behaviour. In his [paper](https://www.nature.com/articles/nn.3658), Ostojic describes a different dynamical state, which can be observed for $g$ and $\eta$ such that the network is in an inhibition dominated regime while increasing the EPSP to values ~ $1\mathrm{mV}$.
How does the spiking behaviour, the rate and the correlation function differ for the classical asynchronous state? What does it imply? For inspiration, you might have a look at Figure 1 d) of the paper.

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
params['g'] = 5
params['eta'] = 2
# Set params['J']
params['rec_start'] = 600.
params['rec_stop'] = 800.

## Your code here

In [ ]:
nest.ResetKernel()
nest.SetKernelStatus({'local_num_threads': 4})
params['g'] = 5.
params['eta'] = 2.
# Set params['J']
params['rec_start'] = 300.
params['rec_stop'] = 4300.

## Your code here

In [ ]:
average_cc(times, cc, n_rec_corr_ex=n_rec_corr_ex, n_rec_corr_in=n_rec_corr_in)